Кустарная тетрадка с наглядным примером по запуску и тестированию стратегий

ссылка{https://kernc.github.io/backtesting.py/} на документацию

ссылка{https://youtu.be/e4ytbIm2Xg0} на видеоурок по библиотечке

ссылка{https://youtu.be/xljQpeYQYkI} на туториал по использованию кастомных индикаторов

In [24]:
# !pip install backtesting
# !pip install python_binance
# !pip install pandas

## Выгрузка данных

In [1]:
from binance import Client
import pandas as pd

In [7]:
INTERVAL_1MINUTE = "1m"
symbol = "ETHUSDT"
end_date = "10 hours ago UTC"

In [4]:
def get_actual_data(
    symbol: str,
    time_frame: str = INTERVAL_1MINUTE,
    end_date: str = "10 hours ago UTC",
) -> pd.DataFrame:
    client = Client()
    df = pd.DataFrame(client.get_historical_klines(symbol, time_frame, end_date))

    df = df.iloc[:, 0:6]
    df.columns = ["Time", "Open", "High", "Low", "Close", "Volume"]
    df.set_index("Time", inplace=True)
    df.index = pd.to_datetime(df.index, unit="ms")
    df = df.astype(float)
    return df

In [9]:
data = get_actual_data(symbol)

In [11]:
data

,Open,High,Low,Close,Volume
Time,,,,,
2023-06-20 22:30:00,1783.34,1783.63,1782.74,1782.75,102.2938
2023-06-20 22:31:00,1782.74,1783.26,1782.74,1783.25,51.3081
2023-06-20 22:32:00,1783.26,1783.26,1782.37,1782.37,110.0553
2023-06-20 22:33:00,1782.36,1783.59,1782.00,1782.78,354.0705
2023-06-20 22:34:00,1782.77,1782.78,1782.57,1782.77,54.2370
...,...,...,...,...,...
2023-06-21 08:25:00,1810.19,1810.19,1808.15,1808.45,210.0973
2023-06-21 08:26:00,1808.46,1810.00,1808.45,1809.99,219.2413
2023-06-21 08:27:00,1809.99,1810.00,1808.74,1808.75,117.1270


## Стратегия

In [12]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

In [13]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

## Бектест

In [15]:
from backtesting import Backtest

In [18]:
cash = 100000
commission = .0001
bt_sma = Backtest(data, SmaCross, cash=cash, commission=commission)
output = bt_sma.run()

In [19]:
output

Start                     2023-06-20 22:30:00
End                       2023-06-21 08:29:00
Duration                      0 days 09:59:00
Exposure Time [%]                   91.833333
Equity Final [$]                 98665.602992
Equity Peak [$]                      100000.0
Return [%]                          -1.334397
Buy & Hold Return [%]                1.450568
Return (Ann.) [%]                  -69.837828
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.057357
Avg. Drawdown [%]                   -2.057357
Max. Drawdown Duration        0 days 09:11:00
Avg. Drawdown Duration        0 days 09:11:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                       -1.33493
Worst Trade [%]                      -1.33493
Avg. Trade [%]                    

In [22]:
bt_sma.plot()

Row(id='1449', ...)